In [1]:
import os

os.environ["WANDB_PROJECT"] = "hidden_capacity_reasoning"
from transformers import Qwen2ForCausalLM, Qwen2Model, AutoTokenizer, BitsAndBytesConfig
import torch
from trl import (
    ModelConfig,
    ScriptArguments,
    SFTConfig,
    SFTTrainer,
    TrlParser,
    get_kbit_device_map,
)

from datasets import load_dataset
from tqdm import tqdm
from hidden_capacity_reasoning.utils import (
    generate_train_examples,
    pad_train_examples,
    tokenize_single_turn,
)
from datasets import Dataset
import gc
import types

# need for auto SFTTrainer patch(possible increase speed)
from unsloth import is_bfloat16_supported
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training,
)
from hidden_capacity_reasoning.utils import (
    EOS_TOKEN_ID,
    TEXT_TOKEN_ID,
    WINDOW_SIZE,
    find_all_linear_names_v2,
)

import time
from datetime import datetime


from hidden_capacity_reasoning.models import (
    Qwen2ForCausalLMCompressionV1,
    Qwen2ModelEmbedPoolerV1,
)


# def main():
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
model = Qwen2ForCausalLMCompressionV1.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map={"": 0},
    attn_implementation="flash_attention_2",
)
device = "cuda"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model.model.requires_grad_(False)

temp_model = Qwen2ModelEmbedPoolerV1.from_pretrained(
    model_name,
    attn_implementation="flash_attention_2",
    torch_dtype=torch.bfloat16,
    device_map={"": 0},
)
print(model.embed_pooler.load_state_dict(temp_model.state_dict()))
temp_model = temp_model.cpu()
del temp_model
gc.collect()
torch.cuda.empty_cache()

dataset = load_dataset("dim/open_orca_905_DeepSeek-R1-Distill-Qwen-1.5B")
dataset = dataset["train"]
dataset = dataset.train_test_split(test_size=10, seed=42)

# test pass
tokenize_single_turn(
    question=dataset["train"][0]["question"],
    answer=dataset["train"][0]["answer"],
    tokenizer=tokenizer,
)
train_examples = [
    tokenize_single_turn(tokenizer=tokenizer, **item)
    for item in tqdm(dataset["train"].to_list()[:3])
]

prepared_train_examples = []
for item in tqdm(train_examples):
    for example in generate_train_examples(
        dataset_batch=[item],
        window_size=WINDOW_SIZE,
    ):
        prepared_train_examples.append(example)

print(
    "max_len",
    max([len(item["original_tokens"]) for item in prepared_train_examples]),
)

dataset = Dataset.from_list(prepared_train_examples)
print(dataset)


def collate_fn(batch):
    padded_batch = pad_train_examples(
        train_examples=batch,
        tokenizer=tokenizer,
    )
    padded_batch = {
        "replaced_original_tokens": padded_batch["replaced_original_tokens"][
            "input_ids"
        ],
        "compressed_input_ids": padded_batch["compressed_input_ids"]["input_ids"],
        "original_tokens": padded_batch["original_tokens"]["input_ids"],
        "attention_mask": padded_batch["compressed_input_ids"]["attention_mask"],
        "labels": padded_batch["compressed_input_ids"]["input_ids"],
    }
    for key in padded_batch.keys():
        padded_batch[key] = torch.tensor(padded_batch[key])
    skip_ids = [TEXT_TOKEN_ID, EOS_TOKEN_ID]
    for skip_id in skip_ids:
        padded_batch["labels"][padded_batch["labels"] == skip_id] = -100
    # print(padded_batch)
    return padded_batch


peft_config = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0.0,
    target_modules=find_all_linear_names_v2(model=model),
)

formatted_date = datetime.fromtimestamp(time.time()).strftime("%Y_%m_%d_%H_%M_%S_%f")
peft_model = get_peft_model(model, peft_config)
peft_model.print_trainable_parameters()

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    data_collator=collate_fn,
    peft_config=peft_config,
    args=SFTConfig(
        per_device_train_batch_size=8,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        num_train_epochs=50,  # Set this for 1 full training run.
        # max_steps=10000,
        learning_rate=1e-4,
        bf16=model.dtype == torch.bfloat16,
        fp16=model.dtype == torch.float16,
        logging_steps=8,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir=f"outputs/{formatted_date}",
        # report_to="wandb",
        report_to="none",
        remove_unused_columns=False,
        dataset_kwargs={"skip_prepare_dataset": True},
        gradient_checkpointing=True,
        save_steps=10000,
        run_name=formatted_date,
    ),
)
trainer.train()

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


Some weights of Qwen2ForCausalLMCompressionV1 were not initialized from the model checkpoint at deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B and are newly initialized: ['embed_pooler.model.layers.0.input_layernorm.weight', 'embed_pooler.model.layers.0.mlp.down_proj.weight', 'embed_pooler.model.layers.0.mlp.gate_proj.weight', 'embed_pooler.model.layers.0.mlp.up_proj.weight', 'embed_pooler.model.layers.0.post_attention_layernorm.weight', 'embed_pooler.model.layers.0.self_attn.k_proj.bias', 'embed_pooler.model.layers.0.self_attn.k_proj.weight', 'embed_pooler.model.layers.0.self_attn.o_proj.weight', 'embed_pooler.model.layers.0.self_attn.q_proj.bias', 'embed_pooler.model.layers.0.self_attn.q_proj.weight', 'embed_pooler.model.layers.0.self_attn.v_proj.bias', 'embed_pooler.model.layers.0.self_attn.v_proj.weight', 'embed_pooler.model.layers.1.input_layernorm.weight', 'embed_pooler.model.layers.1.mlp.down_proj.weight', 'embed_pooler.model.layers.1.mlp.gate_proj.weight', 'embed_pooler.model.layers

<All keys matched successfully>


100%|██████████| 3/3 [00:00<00:00, 2053.68it/s]
/tmp/ipykernel_777457/1449679878.py:142: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


max_len 575
Dataset({
    features: ['replaced_original_tokens', 'compressed_input_ids', 'original_tokens'],
    num_rows: 133
})
trainable params: 18,464,768 || all params: 3,105,893,376 || trainable%: 0.5945


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss
8,1.973200
16,1.885600
24,1.632300
32,1.677700
40,1.608000
48,1.560700
56,1.499900
64,1.470300
72,1.451600
80,1.504600


TrainOutput(global_step=200, training_loss=1.4875512647628784, metrics={'train_runtime': 409.0787, 'train_samples_per_second': 16.256, 'train_steps_per_second': 0.489, 'total_flos': 0.0, 'train_loss': 1.4875512647628784})

In [ ]:
trainer.model

In [ ]:
for name, param in trainer.model.named_parameters():
    if param.requires_grad:
        print(f"Layer: {name}, Requires Gradient: {param.requires_grad}")

In [3]:
model = trainer.model

In [ ]:
dataset

In [2]:
dataset = load_dataset("dim/open_orca_905_DeepSeek-R1-Distill-Qwen-1.5B")
dataset = dataset["train"]
dataset = dataset.train_test_split(test_size=10, seed=42)

In [3]:
model = trainer.model
# prompt = "how many wings has a bird?"
prompt = dataset["train"].to_list()[:5][0]["question"]
messages = [
    # {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt},
]
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
print(text)
model_inputs = tokenizer([text], return_tensors="pt").to(device)

with torch.no_grad():
    # generated_ids = model.generate(
    #     model_inputs.input_ids,
    #     max_new_tokens=1,
    #     do_sample=False,
    # )
    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=1000,
        do_sample=False,
    )
generated_ids = [
    output_ids[len(input_ids) :]
    for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
response

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<｜begin▁of▁sentence｜><｜User｜>Choose your answer from: (A). No (B). Yes
Given those answer options, answer the question: Question: what date did the american civil war start? Would "In 1863, Robert E. Lee's Confederate incursion north ended at the Battle of Gettysburg ." be a reasonable answer?
A:<｜Assistant｜><think>



'Okay, so I need to figure out whether the statement "In 1863, Robert E. Lee\'s Confederate incursion north ended at the Battle of Gettysburg." is a reasonable answer to the question "What date did the American Civil War start?" \n\nFirst, I should recall when the American Civil War actually began. I remember that the Civil War started in 1861 when states seceded from the Union to form the Confederate states. So, the start date is 1861.\n\nNow, looking at the statement, it mentions the Confederate incursion north in 1863. I know that the Union had a major battle in 1863 called the Battle of Gettysburg, which was a significant turning point. That battle was a major conflict between the Union and the Confederacy, and it\'s often cited as the turning point in the Civil War.\n\nSo, the statement is saying that the Confederate incursion ended at Gettysburg in 1863. That makes sense because Gettysburg was a key battle in the Civil War, and it\'s often associated with the end of the Confedera

In [4]:
generated_tokens = tokenizer.apply_chat_template(
    [
        # {"role": "user", "content": "2+2*10"},
        {"role": "user", "content": dataset["train"].to_list()[:5][0]["question"]},
    ],
    tokenize=True,
    add_generation_prompt=True,
)
generated_tokens = torch.tensor(generated_tokens).unsqueeze(0).cuda()
generated_embeds = model.get_input_embeddings()(generated_tokens)
max_steps = 400
for _ in range(max_steps):
    logits = model(
        inputs_embeds=generated_embeds,
    ).logits
    top_token = logits.argmax(-1)[-1][-1]
    top_token_embed = model.get_input_embeddings()(top_token)
    # print(top)
    generated_tokens = torch.cat([generated_tokens, top_token.reshape(1, 1)], dim=1)
    generated_embeds = torch.cat(
        [generated_embeds, top_token_embed.reshape(1, 1, -1)], dim=1
    )
    # break
print(tokenizer.decode(generated_tokens[-1]))
# break
embeds_generation_tokens = generated_tokens[-1]

<｜begin▁of▁sentence｜><｜User｜>Choose your answer from: (A). No (B). Yes
Given those answer options, answer the question: Question: what date did the american civil war start? Would "In 1863, Robert E. Lee's Confederate incursion north ended at the Battle of Gettysburg ." be a reasonable answer?
A:<｜Assistant｜><think>
Okay, so I need to figure out whether the statement "In 1863, Robert E. Lee's Confederate incursion north ended at the Battle of Gettysburg." is a reasonable answer to the question: "What date did the American Civil War start?" 

First, I should recall when the American Civil War actually started. I remember that the Civil War began in 1861 when states seceded from the Union to form the Confederate states. So the start date is 1861.

Now, looking at the statement, it mentions the year 1863. That's two years after the Civil War started. I think the key here is to check if the statement is accurate regarding the end of the Confederate incursion and the Battle of Gettysburg.



In [9]:
from hidden_capacity_reasoning.utils import WINDOW_SIZE
model = trainer.model
generated_tokens = tokenizer.apply_chat_template(
    [
        # {"role": "user", "content": "2+2*10"},
        # {"role": "user", "content": "how many wings has a bird?"},
        {"role": "user", "content": dataset["train"].to_list()[:5][0]["question"]},
    ],
    tokenize=True,
    add_generation_prompt=True,
)

generated_tokens = torch.tensor(generated_tokens).unsqueeze(0).cuda()
generated_embeds = model.get_input_embeddings()(generated_tokens)
max_steps = 50
temp_gen_size = 0
compression_started = False
window_size = WINDOW_SIZE

for step in range(max_steps):
    if temp_gen_size == window_size:
        print(
            "TOKENS FOR EMDED",
            tokenizer.decode(generated_tokens[:, -WINDOW_SIZE:].cpu().tolist()[0]),
        )
        new_embeds_for_compression = generated_embeds[:, -window_size:]
        if hasattr(model.base_model, "embed_pooler"):
            compressed_part = model.base_model.embed_pooler(new_embeds_for_compression)
        else:
            compressed_part = model.embed_pooler(new_embeds_for_compression)

        generated_embeds = torch.cat([generated_embeds, compressed_part], dim=1)
        temp_gen_size = 1

    logits = model(
        inputs_embeds=generated_embeds,
    ).logits
    top_token = logits.argmax(-1)[-1][-1]
    top_token_embed = model.get_input_embeddings()(top_token)
    # print(top)
    generated_tokens = torch.cat([generated_tokens, top_token.reshape(1, 1)], dim=1)

    generated_embeds = torch.cat(
        [generated_embeds, top_token_embed.reshape(1, 1, -1)], dim=1
    )
    print(temp_gen_size, tokenizer.decode(generated_tokens[-1]))

    temp_gen_size += 1

# print(tokenizer.decode(generated_tokens[-1]))

# break

0 <｜begin▁of▁sentence｜><｜User｜>Choose your answer from: (A). No (B). Yes
Given those answer options, answer the question: Question: what date did the american civil war start? Would "In 1863, Robert E. Lee's Confederate incursion north ended at the Battle of Gettysburg ." be a reasonable answer?
A:<｜Assistant｜><think>
Okay
1 <｜begin▁of▁sentence｜><｜User｜>Choose your answer from: (A). No (B). Yes
Given those answer options, answer the question: Question: what date did the american civil war start? Would "In 1863, Robert E. Lee's Confederate incursion north ended at the Battle of Gettysburg ." be a reasonable answer?
A:<｜Assistant｜><think>
Okay,
2 <｜begin▁of▁sentence｜><｜User｜>Choose your answer from: (A). No (B). Yes
Given those answer options, answer the question: Question: what date did the american civil war start? Would "In 1863, Robert E. Lee's Confederate incursion north ended at the Battle of Gettysburg ." be a reasonable answer?
A:<｜Assistant｜><think>
Okay, so
3 <｜begin▁of▁sentenc

In [10]:
print(tokenizer.decode(generated_tokens[-1]))

<｜begin▁of▁sentence｜><｜User｜>Choose your answer from: (A). No (B). Yes
Given those answer options, answer the question: Question: what date did the american civil war start? Would "In 1863, Robert E. Lee's Confederate incursion north ended at the Battle of Gettysburg ." be a reasonable answer?
A:<｜Assistant｜><think>
Okay, so I need to figure out whether the is a reasonable answer to the question about when the American Civil War started. The question is: "What date did the American Civil War start?" And the proposed answer is: "In 18


In [ ]:
# Okay, so I need to figure out whether the statement "In 1863, Robert E. Lee's Confederate incursion north ended at the Battle of Gettysburg." is a reasonable answer to the question: "What date did the American Civil War start?"
# Okay, so I need to figure out whether the is a reasonable answer to the question about when the American Civil War started. The question is: "What date did the American Civil War start?" And the proposed answer is: "In 18